In [56]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockaing data:')
# print(df)
global_df

Global blockaing data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash,blocks_per_day,block_reward_btc,daily_emmisions_btc
BCH,525.0000,3.125000,3.391051e-03,SHA-256,3243979714714552380,164.571429,0.010597,1.743969
BTC,566.0000,3.148082,1.000000e+00,SHA-256,866333381626414840989,152.650177,3.148082,480.555217
DOGE,62.0000,10000.000000,2.560000e-06,Scrypt,2414501045873737,1393.548387,0.025600,35.674839
KAS,0.9999,65.454907,9.199571e-07,kHeavyHash,1362071354782846284,86408.640864,0.000060,5.203157
LTC,148.0000,6.250000,1.207000e-03,Scrypt,2213508382450659,583.783784,0.007544,4.403919
RVN,60.0000,2500.000000,1.440766e-07,KawPow,6506688747348,1440.000000,0.000360,0.518676


In [57]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
market_df = pd.DataFrame(index=index)
market_df['hashrate'] = algo_data_df['speed'] * factors_data_df['mining_factor'] # H/s
market_df['paying'] = algo_data_df['paying'] / sats2btc # BTC/H/day
market_df['daily_consumption_btc'] = market_df.hashrate * market_df.paying
print('Nicehash marketplace data:')
# print(market_data_df)
market_df

Nicehash marketplace data:


,hashrate,paying,daily_consumption_btc
SCRYPT,1.004237e+14,1.658128e-14,1.665154
SHA256ASICBOOST,6.981541e+18,5.472138e-19,3.820396
KAWPOW,7.100380e+11,7.974489e-14,0.056622
KHEAVYHASH,5.441241e+16,3.734210e-18,0.203187


In [58]:
min_order_amount_btc = 5e-3
max_market_utilisation = 0.76

limits_df = pd.DataFrame(index=market_df.index)
limits_df['max_hashrate'] = market_df.hashrate * max_market_utilisation
limits_df['max_daily_consumption_btc'] = market_df.daily_consumption_btc * max_market_utilisation
limits_df['max_hourly_consumption_btc'] = limits_df.max_daily_consumption_btc / 24
limits_df['max_hourly_consumption_dollar'] = round(limits_df.max_hourly_consumption_btc * btc_exchange_rate, 2)
limits_df

,max_hashrate,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_dollar
SCRYPT,7.632202e+13,1.265517,0.052730,5149.57
SHA256ASICBOOST,5.305971e+18,2.903501,0.120979,11814.76
KAWPOW,5.396289e+11,0.043033,0.001793,175.11
KHEAVYHASH,4.135343e+16,0.154422,0.006434,628.37


In [59]:
reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['global_hashrate'] = global_df['nethash']
reachable_df.loc['BCH', 'nh_hashrate'] = limits_df.loc['SHA256ASICBOOST', 'max_hashrate']
reachable_df.loc['BTC', 'nh_hashrate'] = limits_df.loc['SHA256ASICBOOST', 'max_hashrate']
reachable_df.loc['DOGE', 'nh_hashrate'] = limits_df.loc['SCRYPT', 'max_hashrate']
reachable_df.loc['LTC', 'nh_hashrate'] = limits_df.loc['SCRYPT', 'max_hashrate']
reachable_df.loc['KAS', 'nh_hashrate'] = limits_df.loc['KHEAVYHASH', 'max_hashrate']
reachable_df.loc['RVN', 'nh_hashrate'] = limits_df.loc['KAWPOW', 'max_hashrate']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate'] / reachable_df['global_hashrate']  
reachable_df
#limits_df.loc['SHA256ASICBOOST', ['max_hashrate']]

,algorithm,global_hashrate,nh_hashrate,nh_to_global_ratio
BCH,SHA-256,3243979714714552380,5.305971e+18,1.635636
BTC,SHA-256,866333381626414840989,5.305971e+18,0.006125
DOGE,Scrypt,2414501045873737,7.632202e+13,0.03161
KAS,kHeavyHash,1362071354782846284,4.135343e+16,0.030361
LTC,Scrypt,2213508382450659,7.632202e+13,0.03448
RVN,KawPow,6506688747348,5.396289e+11,0.082934
